In [1]:
import cv2
from pathlib import Path
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, GlobalMaxPooling2D, Dropout, Activation, Flatten
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
img_rows, img_cols = 224, 224 

model = Xception(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))


In [3]:
# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False

In [4]:
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [5]:
def add(BottomModel, Class):
    

    top_model = BottomModel.output
    top_model = GlobalMaxPooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(Class,activation='softmax')(top_model)
    return top_model

In [6]:
Class = 2

In [7]:
top = add(model, Class)
model = Model(inputs = model.input, outputs = top)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [9]:
#Image Data Generator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'C://Users//kevin//Documents//Project//Task_4//Train//',
        target_size=(img_rows, img_cols),
        batch_size=64,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        'C://Users//kevin//Documents//Project//Task_4//Validation//',
        target_size=(img_rows, img_cols),
        batch_size=64,
        class_mode='categorical')

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#validation_steps < validation_dataset_size / batch_size
model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator)

Epoch 1/5
4/4 [==============================] - 29s 7s/step - loss: 0.2491 - accuracy: 0.8400 - val_loss: 0.0982 - val_accuracy: 0.9850
Epoch 2/5
4/4 [==============================] - 29s 7s/step - loss: 2.2113 - accuracy: 0.8200 - val_loss: 0.1184 - val_accuracy: 0.9850
Epoch 3/5
4/4 [==============================] - 33s 8s/step - loss: 0.1677 - accuracy: 0.9750 - val_loss: 0.0346 - val_accuracy: 0.9950
Epoch 4/5
4/4 [==============================] - 29s 7s/step - loss: 5.7118e-04 - accuracy: 1.0000 - val_loss: 3.0576e-07 - val_accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 34s 9s/step - loss: 0.0560 - accuracy: 0.9950 - val_loss: 1.7881e-09 - val_accuracy: 1.0000


In [11]:
model.save('Face Detector.h5')